<a href="https://colab.research.google.com/github/steimel60/ML/blob/main/DeepLearning/RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.text.all import *

     |████████████████████████████████| 719 kB 19.2 MB/s 
     |████████████████████████████████| 4.2 MB 46.8 MB/s 
     |████████████████████████████████| 197 kB 54.2 MB/s 
     |████████████████████████████████| 1.2 MB 69.2 MB/s 
     |████████████████████████████████| 346 kB 69.1 MB/s 
     |████████████████████████████████| 60 kB 7.3 MB/s 
     |████████████████████████████████| 1.1 MB 32.1 MB/s 
     |████████████████████████████████| 86 kB 7.4 MB/s 
     |████████████████████████████████| 212 kB 65.3 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 140 kB 70.5 MB/s 
     |████████████████████████████████| 596 kB 53.0 MB/s 
     |████████████████████████████████| 127 kB 80.2 MB/s 
     |████████████████████████████████| 271 kB 75.3 MB/s 
     |████████████████████████████████| 144 kB 51.6 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 112 kB 3.3 MB/s 
     |█████████████████

In [3]:
path = untar_data(URLs.IMDB)
files = get_text_files(path, folders = ['train', 'test', 'unsup'])
txt = files[0].open().read()
txt [:75]

'"The Secret Garden" is one of those rare films that forces you to see Life\''

In [4]:
spacy = WordTokenizer()
toks = first(spacy([txt])) #fastai's tokenizers take collections of docs so we wrap txt in a list
print(coll_repr(toks, 30)) #Show our collection of tokens

(#194) ['"','The','Secret','Garden','"','is','one','of','those','rare','films','that','forces','you','to','see','Life',"'s",'inherent','beauty','.','It','is','a','gothic','tale','which','miraculously','transforms','itself'...]


In [5]:
#We can use fastai to add some additional functionality, like add prefix xx to words without a common base
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt)),31)

(#210) ['xxbos','"','the','xxmaj','secret','xxmaj','garden','"','is','one'...] 31


Subtokens can be used to get the most commonly occuring groups of letters.

In [7]:
txts = L(o.open().read() for o in files[:2000])

def subword(sz):
  sp = SubwordTokenizer(vocab_sz=sz)
  sp.setup(txts)
  return ' '.join(first(sp([txt]))[:40])

subword(1000) #1000 most common combinations of letters (from our txts docs) grouped together and applied to txt example

'▁" The ▁S e c re t ▁G ard en " ▁is ▁one ▁of ▁those ▁ra re ▁films ▁that ▁for ce s ▁you ▁to ▁see ▁Li f e \' s ▁in h er ent ▁be a ut y . ▁It'

Numericalization is the process of mapping tokens to integers

In [8]:
toks200 = txts[:200].map(tkn) #Get small subset of tokenized docs
toks200[0]

(#210) ['xxbos','"','the','xxmaj','secret','xxmaj','garden','"','is','one'...]

In [9]:
#Pass this to set up vocab
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2040) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','it','in','i'...]"

In [10]:
#we can use num as a func
nums = num(toks)[:20]
nums

TensorText([ 22,   0,   0,   0,  22,  16,  41,  14, 155,   0, 142,  20,   0,  42,  15,  94,   0,  23,   0, 737])

In [11]:
#nums is just a tensor of indices we can map to get our text
' '.join(num.vocab[o] for o in nums)

'" xxunk xxunk xxunk " is one of those xxunk films that xxunk you to see xxunk \'s xxunk beauty'